## Low-Level Features vs Cross-Over Points

Using machine learning models to predict cross-over bitrates  from low-level video features extracted from uncompressed videos.

### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import sklearn
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor

import os, sys, warnings
import pickle
from tqdm import tqdm
warnings.filterwarnings("ignore")
sys.path.append("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity")
import bitrate_ladder_construction.BL_functions.dataset_evaluation_functions as dataset_evaluation_functions
import defaults

Paths and Parameters

In [2]:
# Paths
rq_points_dataset_path = defaults.rq_points_dataset_path
features_path = defaults.llf_features_path

# Features
features_set = []
for features_subset in [defaults.glcm_features, defaults.tc_features, defaults.si_features, defaults.ti_features, defaults.cti_features, defaults.cf_features, defaults.ci_features, defaults.dct_features]:
	for f in features_subset:
		if "max" in f:
			continue
		features_set.append(f)
	
# Files
train_video_filenames = defaults.Train_Video_Titles
valid_video_filenames = defaults.Valid_Video_Titles
test_video_filenames = defaults.Test_Video_Titles

# Parameters
Resolutions = defaults.resolutions
CRFs = defaults.CRFs

### Cross-Over Bitrate-1

Dataset

In [3]:
# Training, Validation and Test Datasets
X_train_1, y_train_1 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=train_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(3840,2160),
	low_res=(2560,1440)
)
X_valid_1, y_valid_1 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	temporal_low_level_features=False,
	video_filenames=valid_video_filenames,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(3840,2160),
	low_res=(2560,1440)
)
X_test_1, y_test_1 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=test_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(3840,2160),
	low_res=(2560,1440)
)

# Assertions
assert (X_train_1.shape[1] == X_valid_1.shape[1] == X_test_1.shape[1]) and (y_train_1.shape[1] == y_valid_1.shape[1] == y_test_1.shape[1]), "No.of features do not match."
print ("Dimensions of training dataset = ", X_train_1.shape, y_train_1.shape)
print ("Dimensions of validation dataset = ", X_valid_1.shape, y_valid_1.shape)
print ("Dimensions of test dataset = ", X_test_1.shape, y_test_1.shape)

Dimensions of training dataset =  (70, 93) (70, 1)
Dimensions of validation dataset =  (10, 93) (10, 1)
Dimensions of test dataset =  (30, 93) (30, 1)


Recursive Feature Elimination

In [4]:
# Recursive Feature Elimination
rfe_1 = RFE(estimator=RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2"), n_features_to_select=9)
rfe_1.fit(X_train_1, y_train_1)
indices = [i for i,v in enumerate(rfe_1.support_) if v]

# Features Selected
f = features_set
print ([f[i] for i in indices])

# Feature-Elimination on Inputs
X_train_1 = X_train_1[...,indices]
X_valid_1 = X_valid_1[...,indices]
X_test_1 = X_test_1[...,indices]

['mean_GLCM_energy_mean', 'mean_GLCM_homogeneity_mean', 'mean_SI_mean', 'kurt_TI_std', 'skew_CF', 'skew_CI_V_mean', 'mean_E_Y', 'mean_h_Y', 'mean_h_V']


Training Models

In [5]:
# # Extra-Trees Regressor
# print ("Extra-Trees Regressor:")
# ETR_model_1 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# ETR_model_1.fit(X_train_1, y_train_1)

# print ("MSE =", sklearn.metrics.mean_squared_error(ETR_model_1.predict(X_valid_1), y_valid_1))
# print ("PLCC =", scipy.stats.pearsonr(ETR_model_1.predict(X_valid_1), y_valid_1)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(ETR_model_1.predict(X_valid_1), y_valid_1)[0])
# print ()


# # XG-Boost
# print ("XG-Boost Regressor:")
# XGB_model_1 = XGBRegressor(n_estimators=1250, learning_rate=0.0075, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# XGB_model_1.fit(X_train_1,y_train_1)

# print ("MSE =", sklearn.metrics.mean_squared_error(XGB_model_1.predict(X_valid_1), y_valid_1))
# print ("PLCC =", scipy.stats.pearsonr(XGB_model_1.predict(X_valid_1), y_valid_1)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(XGB_model_1.predict(X_valid_1), y_valid_1)[0])
# print ()


# # Random Forest
# print ("Random-Forest Regressor:")
# RF_model_1 = RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# RF_model_1.fit(X_train_1, y_train_1)

# print ("MSE =", sklearn.metrics.mean_squared_error(RF_model_1.predict(X_valid_1), y_valid_1))
# print ("PLCC =", scipy.stats.pearsonr(RF_model_1.predict(X_valid_1), y_valid_1)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(RF_model_1.predict(X_valid_1), y_valid_1)[0])
# print ()

Testing the best model

In [6]:
# Re-training best model on best features
test_model_1 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
test_model_1.fit(X_train_1, y_train_1)

# Validation
print ("Validation:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_1.predict(X_valid_1), y_valid_1), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_1.predict(X_valid_1), y_valid_1)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_1.predict(X_valid_1), y_valid_1)[0], decimals=3))
print ()

# Testing the model
print ("Testing:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_1.predict(X_test_1), y_test_1), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_1.predict(X_test_1), y_test_1)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_1.predict(X_test_1), y_test_1)[0], decimals=3))

# Saving the weights
pickle.dump(test_model_1, open("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/model_cob_1.pkl", "wb"))
np.save("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/feature_indices_cob_1.npy", indices)

Validation:

MSE = 1.667
PLCC = 0.186
SRCC = 0.079

Testing:

MSE = 2.71
PLCC = 0.305
SRCC = 0.177


### Cross-Over Bitrate-2

Previous Cross-Over Bitrates

In [7]:
train_COB_1 = test_model_1.predict(X_train_1).reshape(-1,1)
valid_COB_1 = test_model_1.predict(X_valid_1).reshape(-1,1)
test_COB_1 = test_model_1.predict(X_test_1).reshape(-1,1)

Dataset

In [8]:
# Training, Validation and Test Datasets
X_train_2, y_train_2 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=train_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(2560,1440),
	low_res=(1920,1080)
)
X_valid_2, y_valid_2 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=valid_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(2560,1440),
	low_res=(1920,1080)
)
X_test_2, y_test_2 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=test_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(2560,1440),
	low_res=(1920,1080)
)

# Concatenating Low-Level Features and Previous Cross-Over Bitrates
X_train_2 = np.concatenate([X_train_2, train_COB_1], axis=1)
X_valid_2 = np.concatenate([X_valid_2, valid_COB_1], axis=1)
X_test_2 = np.concatenate([X_test_2, test_COB_1], axis=1)


# Assertions
assert (X_train_2.shape[1] == X_valid_2.shape[1] == X_test_2.shape[1]) and (y_train_2.shape[1] == y_valid_2.shape[1] == y_test_2.shape[1]), "No.of features do not match."
print ("Dimensions of training dataset = ", X_train_2.shape, y_train_2.shape)
print ("Dimensions of validation dataset = ", X_valid_2.shape, y_valid_2.shape)
print ("Dimensions of test dataset = ", X_test_2.shape, y_test_2.shape)

Dimensions of training dataset =  (70, 94) (70, 1)
Dimensions of validation dataset =  (10, 94) (10, 1)
Dimensions of test dataset =  (30, 94) (30, 1)


Recursive Feature Elimination

In [9]:
# Recursive Feature Elimination
rfe_2 = RFE(estimator=RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2"), n_features_to_select=9)
rfe_2.fit(X_train_2, y_train_2)
indices = [i for i,v in enumerate(rfe_2.support_) if v]

# Features Selected
f = features_set + ["COB_1"]
print ([f[i] for i in indices])

# Feature-Elimination on Inputs
X_train_2 = X_train_2[...,indices]
X_valid_2 = X_valid_2[...,indices]
X_test_2 = X_test_2[...,indices]

['mean_GLCM_contrast_std', 'mean_TC_mean', 'mean_TC_std', 'std_SI_mean', 'mean_TI_std', 'std_TI_mean', 'std_CTI_mean', 'mean_h_U', 'COB_1']


Training Models

In [10]:
# # Extra-Trees Regressor
# print ("Extra-Trees Regressor:")
# ETR_model_2 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# ETR_model_2.fit(X_train_2, y_train_2)


# print ("MSE =", sklearn.metrics.mean_squared_error(ETR_model_2.predict(X_valid_2), y_valid_2))
# print ("PLCC =", scipy.stats.pearsonr(ETR_model_2.predict(X_valid_2), y_valid_2)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(ETR_model_2.predict(X_valid_2), y_valid_2)[0])
# print ()


# # XG-Boost
# print ("XG-Boost Regressor:")
# XGB_model_2 = XGBRegressor(n_estimators=1250, learning_rate=0.0075, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# XGB_model_2.fit(X_train_2,y_train_2)

# print ("MSE =", sklearn.metrics.mean_squared_error(XGB_model_2.predict(X_valid_2), y_valid_2))
# print ("PLCC =", scipy.stats.pearsonr(XGB_model_2.predict(X_valid_2), y_valid_2)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(XGB_model_2.predict(X_valid_2), y_valid_2)[0])
# print ()


# # Random Forest
# print ("Random-Forest Regressor:")
# RF_model_2 = RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# RF_model_2.fit(X_train_2, y_train_2)

# print ("MSE =", sklearn.metrics.mean_squared_error(RF_model_2.predict(X_valid_2), y_valid_2))
# print ("PLCC =", scipy.stats.pearsonr(RF_model_2.predict(X_valid_2), y_valid_2)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(RF_model_2.predict(X_valid_2), y_valid_2)[0])
# print ()

Testing the best model

In [11]:
# Re-training best model on best features
test_model_2 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
test_model_2.fit(X_train_2, y_train_2)

# Validation
print ("Validation:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_2.predict(X_valid_2), y_valid_2), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_2.predict(X_valid_2), y_valid_2)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_2.predict(X_valid_2), y_valid_2)[0], decimals=3))
print ()

# Testing the model
print ("Testing:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_2.predict(X_test_2), y_test_2), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_2.predict(X_test_2), y_test_2)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_2.predict(X_test_2), y_test_2)[0], decimals=3))

# Saving the weights
pickle.dump(test_model_2, open("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/model_cob_2.pkl", "wb"))
np.save("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/feature_indices_cob_2.npy", indices)

Validation:

MSE = 1.811
PLCC = 0.712
SRCC = 0.673

Testing:

MSE = 2.092
PLCC = 0.762
SRCC = 0.816


### Cross-Over Bitrate-3

Previous Cross-Over Bitrates

In [12]:
train_COB_2 = test_model_2.predict(X_train_2).reshape(-1,1)
valid_COB_2 = test_model_2.predict(X_valid_2).reshape(-1,1)
test_COB_2 = test_model_2.predict(X_test_2).reshape(-1,1)

Dataset

In [13]:
# Training, Validation and Test Datasets
X_train_3, y_train_3 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=train_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(1920,1080),
	low_res=(1280,720)
)
X_valid_3, y_valid_3 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=valid_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(1920,1080),
	low_res=(1280,720)
)
X_test_3, y_test_3 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=test_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(1920,1080),
	low_res=(1280,720)
)

# Concatenating Low-Level Features and Previous Cross-Over Bitrates
X_train_3 = np.concatenate([X_train_3, train_COB_1, train_COB_2], axis=1)
X_valid_3 = np.concatenate([X_valid_3, valid_COB_1, valid_COB_2], axis=1)
X_test_3 = np.concatenate([X_test_3, test_COB_1, test_COB_2], axis=1)


# Assertions
assert (X_train_3.shape[1] == X_valid_3.shape[1] == X_test_3.shape[1]) and (y_train_3.shape[1] == y_valid_3.shape[1] == y_test_3.shape[1]), "No.of features do not match."
print ("Dimensions of training dataset = ", X_train_3.shape, y_train_3.shape)
print ("Dimensions of validation dataset = ", X_valid_3.shape, y_valid_3.shape)
print ("Dimensions of test dataset = ", X_test_3.shape, y_test_3.shape)

Dimensions of training dataset =  (70, 95) (70, 1)
Dimensions of validation dataset =  (10, 95) (10, 1)
Dimensions of test dataset =  (30, 95) (30, 1)


Recursive Feature Elimination

In [14]:
# Recursive Feature Elimination
rfe_3 = RFE(estimator=RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2"), n_features_to_select=9)
rfe_3.fit(X_train_3, y_train_3)
indices = [i for i,v in enumerate(rfe_3.support_) if v]

# Features Selected
f = features_set + ["COB_1", "COB_2"]
print ([f[i] for i in indices])

# Feature-Elimination on Inputs
X_train_3 = X_train_3[...,indices]
X_valid_3 = X_valid_3[...,indices]
X_test_3 = X_test_3[...,indices]

['mean_GLCM_homogeneity_std', 'mean_TC_mean', 'mean_TC_std', 'std_SI_mean', 'std_SI_std', 'mean_TI_std', 'std_CTI_mean', 'mean_h_Y', 'COB_2']


Training Models

In [15]:
# # Extra-Trees Regressor
# print ("Extra-Trees Regressor:")
# ETR_model_3 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# ETR_model_3.fit(X_train_3, y_train_3)

# print ("MSE =", sklearn.metrics.mean_squared_error(ETR_model_3.predict(X_valid_3), y_valid_3))
# print ("PLCC =", scipy.stats.pearsonr(ETR_model_3.predict(X_valid_3), y_valid_3)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(ETR_model_3.predict(X_valid_3), y_valid_3)[0])
# print ()


# # XG-Boost
# print ("XG-Boost Regressor:")
# XGB_model_3 = XGBRegressor(n_estimators=1250, learning_rate=0.0075, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# XGB_model_3.fit(X_train_3,y_train_3)

# print ("MSE =", sklearn.metrics.mean_squared_error(XGB_model_3.predict(X_valid_3), y_valid_3))
# print ("PLCC =", scipy.stats.pearsonr(XGB_model_3.predict(X_valid_3), y_valid_3)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(XGB_model_3.predict(X_valid_3), y_valid_3)[0])
# print ()


# # Random Forest
# print ("Random-Forest Regressor:")
# RF_model_3 = RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# RF_model_3.fit(X_train_3, y_train_3)

# print ("MSE =", sklearn.metrics.mean_squared_error(RF_model_3.predict(X_valid_3), y_valid_3))
# print ("PLCC =", scipy.stats.pearsonr(RF_model_3.predict(X_valid_3), y_valid_3)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(RF_model_3.predict(X_valid_3), y_valid_3)[0])
# print ()

Testing the best model

In [16]:
# Re-training best model on best features
test_model_3 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
test_model_3.fit(X_train_3, y_train_3)

# Validation
print ("Validation:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_3.predict(X_valid_3), y_valid_3), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_3.predict(X_valid_3), y_valid_3)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_3.predict(X_valid_3), y_valid_3)[0], decimals=3))
print ()

# Testing the model
print ("Testing:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_3.predict(X_test_3), y_test_3), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_3.predict(X_test_3), y_test_3)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_3.predict(X_test_3), y_test_3)[0], decimals=3))

# Saving the weights
pickle.dump(test_model_3, open("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/model_cob_3.pkl", "wb"))
np.save("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/feature_indices_cob_3.npy", indices)

Validation:

MSE = 2.353
PLCC = 0.566
SRCC = 0.733

Testing:

MSE = 1.615
PLCC = 0.704
SRCC = 0.747


### Cross-Over Bitrate-4

Previous Cross-Over Bitrates

In [17]:
train_COB_3 = test_model_3.predict(X_train_3).reshape(-1,1)
valid_COB_3 = test_model_3.predict(X_valid_3).reshape(-1,1)
test_COB_3 = test_model_3.predict(X_test_3).reshape(-1,1)

Dataset

In [18]:
# Training, Validation and Test Datasets
X_train_4, y_train_4 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=train_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(1280,720),
	low_res=(960,540)
)
X_valid_4, y_valid_4 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=valid_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(1280,720),
	low_res=(960,540)
)
X_test_4, y_test_4 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=test_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(1280,720),
	low_res=(960,540)
)

# Concatenating Low-Level Features and Previous Cross-Over Bitrates
X_train_4 = np.concatenate([X_train_4, train_COB_1, train_COB_2, train_COB_3], axis=1)
X_valid_4 = np.concatenate([X_valid_4, valid_COB_1, valid_COB_2, valid_COB_3], axis=1)
X_test_4 = np.concatenate([X_test_4, test_COB_1, test_COB_2, test_COB_3], axis=1)


# Assertions
assert (X_train_4.shape[1] == X_valid_4.shape[1] == X_test_4.shape[1]) and (y_train_4.shape[1] == y_valid_4.shape[1] == y_test_4.shape[1]), "No.of features do not match."
print ("Dimensions of training dataset = ", X_train_4.shape, y_train_4.shape)
print ("Dimensions of validation dataset = ", X_valid_4.shape, y_valid_4.shape)
print ("Dimensions of test dataset = ", X_test_4.shape, y_test_4.shape)

Dimensions of training dataset =  (70, 96) (70, 1)
Dimensions of validation dataset =  (10, 96) (10, 1)
Dimensions of test dataset =  (30, 96) (30, 1)


Recursive Feature Elimination

In [19]:
# Recursive Feature Elimination
rfe_4 = RFE(estimator=RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2"), n_features_to_select=9)
rfe_4.fit(X_train_4, y_train_4)
indices = [i for i,v in enumerate(rfe_4.support_) if v]

# Features Selected
f = features_set + ["COB_1", "COB_2", "COB_3"]
print ([f[i] for i in indices])

# Feature-Elimination on Inputs
X_train_4 = X_train_4[...,indices]
X_valid_4 = X_valid_4[...,indices]
X_test_4 = X_test_4[...,indices]

['mean_GLCM_correlation_mean', 'std_SI_mean', 'std_SI_std', 'mean_TI_mean', 'mean_TI_std', 'std_CTI_mean', 'mean_h_Y', 'COB_2', 'COB_3']


Training Models

In [20]:
# # Extra-Trees Regressor
# print ("Extra-Trees Regressor:")
# ETR_model_4 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# ETR_model_4.fit(X_train_4, y_train_4)

# print ("MSE =", sklearn.metrics.mean_squared_error(ETR_model_4.predict(X_valid_4), y_valid_4))
# print ("PLCC =", scipy.stats.pearsonr(ETR_model_4.predict(X_valid_4), y_valid_4)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(ETR_model_4.predict(X_valid_4), y_valid_4)[0])
# print ()


# # XG-Boost
# print ("XG-Boost Regressor:")
# XGB_model_4 = XGBRegressor(n_estimators=1250, learning_rate=0.0075, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# XGB_model_4.fit(X_train_4,y_train_4)

# print ("MSE =", sklearn.metrics.mean_squared_error(XGB_model_4.predict(X_valid_4), y_valid_4))
# print ("PLCC =", scipy.stats.pearsonr(XGB_model_4.predict(X_valid_4), y_valid_4)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(XGB_model_4.predict(X_valid_4), y_valid_4)[0])
# print ()


# # Random Forest
# print ("Random-Forest Regressor:")
# RF_model_4 = RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# RF_model_4.fit(X_train_4, y_train_4)

# print ("MSE =", sklearn.metrics.mean_squared_error(RF_model_4.predict(X_valid_4), y_valid_4))
# print ("PLCC =", scipy.stats.pearsonr(RF_model_4.predict(X_valid_4), y_valid_4)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(RF_model_4.predict(X_valid_4), y_valid_4)[0])
# print ()

Testing the best model

In [21]:
# Re-training best model on best features
test_model_4 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
test_model_4.fit(X_train_4, y_train_4)

# Validation
print ("Validation:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_4.predict(X_valid_4), y_valid_4), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_4.predict(X_valid_4), y_valid_4)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_4.predict(X_valid_4), y_valid_4)[0], decimals=3))
print ()

# Testing the model
print ("Testing:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_4.predict(X_test_4), y_test_4), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_4.predict(X_test_4), y_test_4)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_4.predict(X_test_4), y_test_4)[0], decimals=3))

# Saving the weights
pickle.dump(test_model_4, open("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/model_cob_4.pkl", "wb"))
np.save("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/feature_indices_cob_4.npy", indices)

Validation:

MSE = 3.534
PLCC = 0.504
SRCC = 0.539

Testing:

MSE = 1.636
PLCC = 0.625
SRCC = 0.679


### Cross-Over Bitrate-5

Previous Cross-Over Bitrates

In [22]:
train_COB_4 = test_model_4.predict(X_train_4).reshape(-1,1)
valid_COB_4 = test_model_4.predict(X_valid_4).reshape(-1,1)
test_COB_4 = test_model_4.predict(X_test_4).reshape(-1,1)

Dataset

In [23]:
# Training, Validation and Test Datasets
X_train_5, y_train_5 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=train_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(960,540),
	low_res=(768,432)
)
X_valid_5, y_valid_5 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=valid_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(960,540),
	low_res=(768,432)
)
X_test_5, y_test_5 = dataset_evaluation_functions.LowLevelFeatures_CrossOverBitrates_Dataset(
	codec="libx265",
	preset="medium",
	quality_metric="vmaf",
	features_names=features_set,
	video_filenames=test_video_filenames,
	temporal_low_level_features=False,
	Resolutions_Considered=Resolutions,
	CRFs_Considered=CRFs,
	bitrates_Considered=None,
	QPs_Considered=None,
	high_res=(960,540),
	low_res=(768,432)
)

# Concatenating Low-Level Features and Previous Cross-Over Bitrates
X_train_5 = np.concatenate([X_train_5, train_COB_1, train_COB_2, train_COB_3, train_COB_4], axis=1)
X_valid_5 = np.concatenate([X_valid_5, valid_COB_1, valid_COB_2, valid_COB_3, valid_COB_4], axis=1)
X_test_5 = np.concatenate([X_test_5, test_COB_1, test_COB_2, test_COB_3, test_COB_4], axis=1)


# Assertions
assert (X_train_5.shape[1] == X_valid_5.shape[1] == X_test_5.shape[1]) and (y_train_5.shape[1] == y_valid_5.shape[1] == y_test_5.shape[1]), "No.of features do not match."
print ("Dimensions of training dataset = ", X_train_5.shape, y_train_5.shape)
print ("Dimensions of validation dataset = ", X_valid_5.shape, y_valid_5.shape)
print ("Dimensions of test dataset = ", X_test_5.shape, y_test_5.shape)

Dimensions of training dataset =  (70, 97) (70, 1)
Dimensions of validation dataset =  (10, 97) (10, 1)
Dimensions of test dataset =  (30, 97) (30, 1)


Recursive Feature Elimination

In [24]:
# Recursive Feature Elimination
rfe_5 = RFE(estimator=RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2"), n_features_to_select=9)
rfe_5.fit(X_train_5, y_train_5)
indices = [i for i,v in enumerate(rfe_5.support_) if v]

# Features Selected
f = features_set + ["COB_1", "COB_2", "COB_3", "COB_4"]
print ([f[i] for i in indices])

# Feature-Elimination on Inputs
X_train_5 = X_train_5[...,indices]
X_valid_5 = X_valid_5[...,indices]
X_test_5 = X_test_5[...,indices]

['mean_GLCM_correlation_mean', 'mean_SI_mean', 'std_SI_mean', 'mean_TI_std', 'mean_E_Y', 'mean_h_Y', 'COB_2', 'COB_3', 'COB_4']


Training Models

In [25]:
# # Extra-Trees Regressor
# print ("Extra-Trees Regressor:")
# ETR_model_5 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# ETR_model_5.fit(X_train_5, y_train_5)

# print ("MSE =", sklearn.metrics.mean_squared_error(ETR_model_5.predict(X_valid_5), y_valid_5))
# print ("PLCC =", scipy.stats.pearsonr(ETR_model_5.predict(X_valid_5), y_valid_5)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(ETR_model_5.predict(X_valid_5), y_valid_5)[0])
# print ()


# # XG-Boost
# print ("XG-Boost Regressor:")
# XGB_model_5 = XGBRegressor(n_estimators=1250, learning_rate=0.0075, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# XGB_model_5.fit(X_train_5,y_train_5)

# print ("MSE =", sklearn.metrics.mean_squared_error(XGB_model_5.predict(X_valid_5), y_valid_5))
# print ("PLCC =", scipy.stats.pearsonr(XGB_model_5.predict(X_valid_5), y_valid_5)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(XGB_model_5.predict(X_valid_5), y_valid_5)[0])
# print ()


# # Random Forest
# print ("Random-Forest Regressor:")
# RF_model_5 = RandomForestRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
# RF_model_5.fit(X_train_5, y_train_5)

# print ("MSE =", sklearn.metrics.mean_squared_error(RF_model_5.predict(X_valid_5), y_valid_5))
# print ("PLCC =", scipy.stats.pearsonr(RF_model_5.predict(X_valid_5), y_valid_5)[0][0])
# print ("SRCC =", scipy.stats.spearmanr(RF_model_5.predict(X_valid_5), y_valid_5)[0])
# print ()

Testing the best model

In [26]:
# Re-training best model on best features
test_model_5 = ExtraTreesRegressor(n_estimators=1250, criterion="squared_error", random_state=2, max_depth=12, max_features="log2")
test_model_5.fit(X_train_5, y_train_5)

# Validation
print ("Validation:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_5.predict(X_valid_5), y_valid_5), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_5.predict(X_valid_5), y_valid_5)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_5.predict(X_valid_5), y_valid_5)[0], decimals=3))
print ()

# Testing the model
print ("Testing:\n")
print ("MSE =", np.round(sklearn.metrics.mean_squared_error(test_model_5.predict(X_test_5), y_test_5), decimals=3))
print ("PLCC =", np.round(scipy.stats.pearsonr(test_model_5.predict(X_test_5), y_test_5)[0][0], decimals=3))
print ("SRCC =", np.round(scipy.stats.spearmanr(test_model_5.predict(X_test_5), y_test_5)[0], decimals=3))

# Saving the weights
pickle.dump(test_model_5, open("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/model_cob_5.pkl", "wb"))
np.save("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/models/ML/CrossOver_Bitrates/low_level_features/feature_indices_cob_5.npy", indices)

Validation:

MSE = 0.948
PLCC = 0.628
SRCC = 0.697

Testing:

MSE = 0.789
PLCC = 0.697
SRCC = 0.689
